---
# Notebook 1: Costruzione del Database Finanziario
---
### **Autore:** Valerio Lapiello
* **Data Ultima Modifica:** Ottobre 2025
* **Profilo LinkedIn:** [Valerio Lapiello](https://www.linkedin.com/in/valerio-lapiello-597801377/)
* **Repositories GitHub:** [ValeatorioLab](https://github.com/ValeatorioLab?tab=repositories)

---

## Introduzione: La Creazione di un Asset Informativo
Questo notebook documenta la prima e più fondamentale fase del nostro progetto di finanza quantitativa: la costruzione di un **database robusto, pulito e affidabile**. L'obiettivo di questo processo di Data Engineering è creare le fondamenta su cui poggerà l'intera analisi successiva, dalla feature engineering all'addestramento del modello di machine learning e al backtesting della strategia.

Il database, denominato `quant_strategy.db`, è un database relazionale SQLite progettato per essere efficiente, portabile e facilmente interrogabile. È composto da tre tabelle principali, ciascuna con uno scopo specifico:

1. `securities_master` **(Anagrafica Titoli)**: Funge da registro centrale per il nostro universo di investimento. Contiene le informazioni "statiche" di ogni titolo, come il ticker, il nome dell'azienda e il settore di appartenenza. L'universo di investimento è stato definito combinando i componenti degli indici **FTSE MIB** e **FTSE Italia Mid Cap** per ottenere una copertura completa e stabile del mercato azionario italiano.

2. `daily_prices` **(Prezzi Giornalieri)**: È il cuore pulsante del database. Contiene l'intero storico dei prezzi giornalieri (Open, High, Low, Close, Volume) per ogni titolo presente nell'anagrafica. I dati sono stati scaricati a partire dal **2002**, una decisione strategica presa per escludere a priori un periodo storico di dati meno affidabili e affetti da gravi errori di scala.

3. `macro_data` **(Contesto Macroeconomico)**: Questa tabella fornisce il contesto economico e finanziario in cui operano le aziende. Contiene una selezione curata di indicatori macroeconomici chiave, sia per l'Eurozona che specifici per l'Italia, permettendo al nostro futuro modello di comprendere le dinamiche di mercato più ampie.

**Fonti dei Dati**: Per garantire la massima qualità e accessibilità, abbiamo utilizzato due fonti autorevoli:

`yfinance`: Per il download massivo dei dati di prezzo giornalieri.

**FRED (Federal Reserve Economic Data)**: Come fonte primaria e più affidabile per tutte le serie storiche macroeconomiche.

Durante la costruzione, abbiamo affrontato e risolto diverse sfide tipiche del mondo reale, tra cui la gestione di dati storici corrotti, la scelta delle fonti dati più affidabili e l'armonizzazione di serie storiche con frequenze diverse. Ogni scelta metodologica è stata ponderata per garantire che il prodotto finale sia un database di livello professionale, pronto per l'analisi quantitativa.

---

## Indice del Notebook
* [1. Setup dell'Ambiente](#1.)
* [2. Tabella 1: Anagrafica Titoli (`securities_master`)](#2.)
    * [2.1 Definizione dell'Universo di Investimento](#2.1)
    * [2.2 Creazione della Struttura della Tabella](#2.2)
    * [2.3 Popolamento e Validazione dei Dati Anagrafici](#2.3)
    * [2.4 Controllo Iniziale](#2.4)
* [3. Tabella 2: Prezzi Giornalieri (`daily_prices`)](#3.)
    * [3.1 Progettazione della Tabella dei Prezzi](#3.1)
    * [3.2 Popolamento e Quality Assurance dei Dati di Prezzo](#3.2)
    * [3.3 Controllo Iniziale](#3.3)
* [4. Tabella 3: Dati Macroeconomici (`macro_data`)](#4.)
    * [4.1 Selezione degli Indicatori Macroeconomici](#4.1)
    * [4.2 Popolamento e Armonizzazione dei Dati Macro](#4.2)
    * [4.3 Controllo Iniziale](#4.2)
* [5. Quality Assurance Finale del Database](#5.)
    * [5.1 Test di Integrità Logica (Sanity Checks)](#5.1)
    * [5.2 Test di Coerenza Strutturale](#5.2)
* [6. Creazione dello Script di Aggiornamento](#6.)
    * [6.1 Logica e Best Practice](#6.1)
    * [6.2 Codice dello Script `update_database.py`](#6.2)
* [7. Nota Metodologica](#7.)
        
---

<a id="1."></a>
## 1. Setup dell'Ambiente
Iniziamo importando tutte le librerie Python che ci serviranno per interagire con i database, scaricare dati finanziari e manipolare le tabelle. Inoltre definiamo il nome del nostro database

In [1]:
# Import per la gestione del database
import sqlite3

# Import per il download dei dati finanziari
import yfinance as yf
import pandas_datareader.data as pdr

# Import per la manipolazione e l'analisi dei dati
import pandas as pd
import numpy as np

# Import per la gestione delle date e le barre di progresso
import datetime
from tqdm import tqdm

# Import per la visualizzazione grafica
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Nome del file del nostro database SQLite
DB_NAME = 'ita100_stock_and_macro.db'

<a id="2."></a>
## 2. Tabella 1: Anagrafica Titoli (`securities_master`)
<a id="2.1"></a>
### 2.1 Definizione dell'Universo di Investimento
La prima tabella che creiamo è l'anagrafica dei titoli. Per definire il nostro universo di investimento, abbiamo scelto di combinare le aziende degli indici FTSE MIB e FTSE Italia Mid Cap. Questa scelta ci fornisce un paniere di circa 100 titoli, rappresentativo delle aziende a grande e media capitalizzazione del mercato italiano.

In [3]:
# Liste dei ticker che compongono il nostro universo di investimento
ticker_ftse_mib_40 = ["A2A.MI","AMP.MI","AZM.MI","BMED.MI","BMPS.MI","BAMI.MI","BPSO.MI","BPE.MI","BC.MI","BZU.MI",
                      "CPR.MI","DIA.MI","ENEL.MI","ENI.MI","RACE.MI","FBK.MI","G.MI","HER.MI","IP.MI","ISP.MI",
                      "INW.MI","IG.MI","IVG.MI","LDO.MI","LTMC.MI","MB.MI","MONC.MI","NEXI.MI","PST.MI","PRY.MI",
                      "REC.MI","SPM.MI","SRG.MI","STLAM.MI","STMMI.MI","TIT.MI","TEN.MI","TRN.MI","UCG.MI","UNI.MI"]
ticker_ftse_mid_cap = ["ACE.MI","ARN.MI","ANIM.MI","ARIS.MI","ASC.MI","AVIO.MI","BGN.MI","IF.MI","BDB.MI","BFF.MI",
                       "BRE.MI","CED.MI","CRL.MI","CMB.MI","CEM.MI","CIR.MI","COM.MI","CE.MI","DIS.MI","DAN.MI",
                       "DLG.MI","DOV.MI","ELN.MI","ENAV.MI","ERG.MI","YACHT.MI","FCT.MI","GVS.MI","ICOS.MI","IRE.MI",
                       "ITM.MI","JUVE.MI","LUVE.MI","MAIRE.MI","MARR.MI","MFEA.MI","MFEB.MI","MOL.MI","MN.MI","NWL.MI",
                       "OVS.MI","PHN.MI","PHIL.MI","PIA.MI","PIRC.MI","RWAY.MI","RCS.MI","REY.MI","SFL.MI","SFER.MI",
                       "SL.MI","SES.MI","SOL.MI","TIP.MI","TGYM.MI","TPRO.MI","TNXT.MI","WBD.MI","WIIT.MI","ZV.MI"]

<a id="2.2"></a>
### 2.2 Creazione della Struttura della Tabella
Definiamo la struttura (schema) della tabella `securities_master` tramite una query SQL. Seguiamo le best practice:
- `id` **INTEGER PRIMARY KEY**: Una chiave primaria numerica, unica e auto-incrementante, per garantire performance e stabilità.

- `ticker` **TEXT NOT NULL UNIQUE**: La chiave "naturale" del titolo, con un vincolo di unicità per evitare duplicati.

In [4]:
# Connettiamoci al database e creiamo la tabella se non esiste
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS securities_master (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ticker TEXT NOT NULL UNIQUE,
    company_name TEXT,
    sector TEXT,
    index_name TEXT NOT NULL
);
"""

cursor.execute(create_table_query)
conn.commit()
conn.close()
print("Tabella 'securities_master' creata o già esistente.")

Tabella 'securities_master' creata o già esistente.


<a id="2.3"></a>
### 2.3 Popolamento e Validazione dei Dati Anagrafici
Questa funzione itera sulle nostre liste di ticker, contatta `yfinance` per recuperare i dati anagrafici (nome, settore) e li inserisce nella tabella. Durante lo sviluppo, abbiamo incontrato il problema di ticker errati o non più esistenti. Per risolverlo, la funzione è stata resa più robusta:
- **Controllo di Validità**: Prima di inserire i dati, verifica che le informazioni ricevute da `yfinance` siano valide (controllando la presenza del campo 'sector').

- **Inserimento Sicuro**: Utilizza query parametrizzate (`?`) per prevenire errori e attacchi SQL Injection, e l'istruzione `INSERT OR IGNORE` per gestire elegantemente i duplicati.

In [5]:
def populate_securities_master(tickers, index_name):
    """
    Popola la tabella 'securities_master' con i dati anagrafici per una data lista di ticker,
    includendo controlli di validità per scartare ticker non esistenti.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    print(f"\nInizio popolamento per l'indice: {index_name}...")

    for ticker_symbol in tqdm(tickers, desc=f"Popolamento {index_name}"):
        try:
            ticker_info = yf.Ticker(ticker_symbol).info
            if 'sector' in ticker_info and ticker_info['sector'] is not None:
                company_name = ticker_info.get('longName', 'N/A')
                sector = ticker_info.get('sector', 'N/A')
                
                insert_query = "INSERT OR IGNORE INTO securities_master (ticker, company_name, sector, index_name) VALUES (?, ?, ?, ?);"
                cursor.execute(insert_query, (ticker_symbol, company_name, sector, index_name))
            else:
                raise ValueError("Ticker non valido o dati insufficienti")
        except Exception as e:
            print(f"  - ATTENZIONE: Ticker {ticker_symbol} scartato. Motivo: {e}")
            
    conn.commit()
    conn.close()
    
    print(f"Popolamento completato per {index_name}.")

In [6]:
# Eseguiamo la funzione per entrambe le liste
populate_securities_master(ticker_ftse_mib_40, "FTSE MIB")
populate_securities_master(ticker_ftse_mid_cap, "FTSE Mid Cap")

<a id="2.4"></a>
### 2.4 Controllo Iniziale
Verifichiamo subito che il popolamento sia andato a buon fine, visualizzando le prime 10 righe della tabella e assicurandoci che tutti i 100 titoli siano stati inclusi. Per questa operazione di analisi interattiva, utilizziamo l'estensione `jupysql` (precedentemente `ipython-sql`), che ci permette di scrivere SQL direttamente nel notebook.

In [7]:
# Carichiamo l'estensione SQL e ci connettiamo al database
%load_ext sql
%sql sqlite:///ita100_stock_and_macro.db

Connecting to 'sqlite:///ita100_stock_and_macro.db'

In [8]:
%%sql
SELECT * FROM securities_master LIMIT 10;

Running query in 'sqlite:///ita100_stock_and_macro.db'

id,ticker,company_name,sector,index_name
1,A2A.MI,A2A S.p.A.,Utilities,FTSE MIB
2,AMP.MI,Amplifon S.p.A.,Healthcare,FTSE MIB
3,AZM.MI,Azimut Holding S.p.A.,Financial Services,FTSE MIB
4,BMED.MI,Banca Mediolanum S.p.A.,Financial Services,FTSE MIB
5,BMPS.MI,Banca Monte dei Paschi di Siena S.p.A.,Financial Services,FTSE MIB
6,BAMI.MI,Banco BPM S.p.A.,Financial Services,FTSE MIB
7,BPSO.MI,Banca Popolare di Sondrio S.p.A,Financial Services,FTSE MIB
8,BPE.MI,BPER Banca SpA,Financial Services,FTSE MIB
9,BC.MI,Brunello Cucinelli S.p.A.,Consumer Cyclical,FTSE MIB
10,BZU.MI,Buzzi S.p.A.,Basic Materials,FTSE MIB


In [9]:
%%sql
SELECT count(id) FROM securities_master;

Running query in 'sqlite:///ita100_stock_and_macro.db'

count(id)
100


<a id="3"></a>
## 3. Tabella 2: Prezzi Giornalieri (`daily_prices`)
Questa tabella è il cuore del nostro database e conterrà la materia prima per tutta l'analisi quantitativa: i dati storici dei prezzi dei titoli. La sua corretta progettazione e un popolamento robusto sono cruciali per l'affidabilità dell'intero progetto.
<a id="3.1"></a>
### 3.1 Progettazione della Tabella dei Prezzi
Abbiamo progettato la tabella `daily_prices` per archiviare i dati alla loro massima granularità disponibile, ovvero quella **giornaliera**. Questa scelta ci garantisce la massima flessibilità per calcoli futuri (rendimenti settimanali, mensili, volatilità, etc.). La struttura include:
- `security_id` **INTEGER NOT NULL, FOREIGN KEY**: Questa colonna collega ogni riga di prezzo all'azione corrispondente nella tabella securities_master, garantendo l'**integrità referenziale** del database.

- **Colonne OHLCV**: I dati standard di mercato (Open, High, Low, Close, Volume).

- Vincolo `UNIQUE`: Una chiave unica sulla coppia (`security_id`, `price_date`) per impedire l'inserimento accidentale di dati duplicati per la stessa azione nello stesso giorno.

In [10]:
# Connettiamoci al database e creiamo la tabella dei prezzi giornalieri
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

create_prices_table_query = """
CREATE TABLE IF NOT EXISTS daily_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    security_id INTEGER NOT NULL,
    price_date TEXT NOT NULL,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    FOREIGN KEY (security_id) REFERENCES securities_master (id),
    UNIQUE (security_id, price_date)
);
"""

cursor.execute(create_prices_table_query)
conn.commit()
conn.close()
print("Tabella 'daily_prices' creata o già esistente.")

Tabella 'daily_prices' creata o già esistente.


<a id="3.2"></a>
### 3.2 Popolamento e Quality Assurance dei Dati di Prezzo
Questa è la fase più critica dell'intero processo di Data Engineering. Durante lo sviluppo, l'analisi iniziale dei dati grezzi ha rivelato la presenza di significative anomalie, tra cui prezzi negativi, valori astronomici dovuti a errati aggiustamenti storici e incongruenze logiche nei dati OHLC.

Per risolvere queste sfide, abbiamo implementato una funzione di popolamento "blindata" che agisce come un vero e proprio "sistema immunitario" per il nostro database:
1. **Azione alla Fonte**: La funzione scarica i dati solo a partire dal `2002-01-01`. Questa decisione strategica esclude a priori un lungo periodo storico in cui i dati si sono rivelati inaffidabili e affetti da errori di scala.

2. **Controlli di Qualità (Sanity Checks)**: Prima di inserire qualsiasi dato, la funzione applica un set rigoroso di filtri per scartare le righe palesemente corrotte:
    * Verifica che i prezzi di chiusura siano positivi.
    * Verifica la coerenza logica dei dati OHLC (es. `High >= Low`, `High >= Close`, etc.).


3. **Download Intelligente**: Il parametro `auto_adjust=True` di `yfinance` viene utilizzato per ottenere prezzi già aggiustati per dividendi e stock split, semplificando i calcoli futuri dei rendimenti totali.

In [11]:
# Definiamo la data di inizio globale per tutti i download di prezzi
start_date_prices = "2002-01-01"

def populate_daily_prices():
    """
    Popola la tabella 'daily_prices' scaricando i dati a partire da una data 
    specifica e applicando un set completo di controlli di validità.
    """
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT id, ticker FROM securities_master")
        securities = cursor.fetchall()
        print(f"Trovate {len(securities)} azioni. Inizio download dati storici dal {start_date_prices}...")
        
        total_rows_removed = 0

        for security in tqdm(securities, desc="Scaricamento Dati Prezzi"):
            security_id, ticker_symbol = security
            try:
                # 1. Download dei Dati a partire dalla data specificata
                price_data = yf.download(
                    ticker_symbol, 
                    start=start_date_prices, 
                    auto_adjust=True, 
                    progress=False
                )
                
                if price_data.empty:
                    continue

                # 2. Controlli di Validità Completi (Sanity Checks)
                original_rows = len(price_data)
                price_data = price_data[price_data['Close'] > 0]
                price_data = price_data[price_data['High'] >= price_data['Low']]
                price_data = price_data[(price_data['High'] >= price_data['Open']) & (price_data['High'] >= price_data['Close'])]
                
                removed_rows = original_rows - len(price_data)
                if removed_rows > 0:
                    total_rows_removed += removed_rows

                # 3. Preparazione e Inserimento dei Dati
                price_data.reset_index(inplace=True)
                price_data.rename(columns={
                    'Date': 'price_date', 'Open': 'open', 'High': 'high',
                    'Low': 'low', 'Close': 'close', 'Volume': 'volume'
                }, inplace=True)
                
                price_data['security_id'] = security_id
                price_data['price_date'] = price_data['price_date'].dt.strftime('%Y-%m-%d')
                
                columns_to_insert = ['security_id', 'price_date', 'open', 'high', 'low', 'close', 'volume']
                data_to_insert = price_data[columns_to_insert]

                insert_query = "INSERT OR IGNORE INTO daily_prices (security_id, price_date, open, high, low, close, volume) VALUES (?, ?, ?, ?, ?, ?, ?);"
                cursor.executemany(insert_query, data_to_insert.values.tolist())
                conn.commit()

            except Exception as e:
                print(f"  - ERRORE durante il processamento di {ticker_symbol}: {e}")
                conn.rollback()
    except sqlite3.Error as e:
        print(f"Errore di database: {e}")
    finally:
        if conn:
            conn.close()
            print(f"\n--- Processo di popolamento completato. Rimosse in totale {total_rows_removed} righe incoerenti. ---")

Ora eseguiamo il processo di popolamento. Per garantire che il nostro database sia sempre pulito e aggiornato, adottiamo un flusso **"Distruggi e Ricostruisci"**:
1. `DROP TABLE`: Eliminiamo la tabella daily_prices se esiste, per rimuovere eventuali dati vecchi.

2. `CREATE TABLE`: Ricreiamo la struttura vuota della tabella.

3. `populate_daily_prices()`: Eseguiamo la funzione per popolarla con i dati freschi e validati.

Questo approccio rende il nostro notebook **idempotente**, ovvero rieseguibile più volte ottenendo sempre lo stesso risultato finale pulito.

In [23]:
# Connettiamoci al database per eseguire le operazioni di pulizia e creazione
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# 1. Eliminiamo la tabella se esiste per garantire una ripartenza pulita
print("Eliminazione della tabella 'daily_prices' esistente...")
cursor.execute("DROP TABLE IF EXISTS daily_prices;")
conn.commit()

# 2. Ricreiamo la tabella con la sua struttura corretta
print("Creazione della nuova tabella 'daily_prices'...")
cursor.execute(create_prices_table_query) # Riutilizziamo la query definita nella Cella 3.1
conn.commit()

# Chiudiamo la connessione prima di chiamare la funzione di popolamento
conn.close()

# 3. Eseguiamo la funzione di popolamento sulla tabella appena creata
populate_daily_prices()

Eliminazione della tabella 'daily_prices' esistente...
Creazione della nuova tabella 'daily_prices'...
Trovate 100 azioni. Inizio download dati storici dal 2002-01-01...


Scaricamento Dati Prezzi: 100%|██████████| 100/100 [00:43<00:00,  2.30it/s]


--- Processo di popolamento completato. Rimosse in totale 0 righe incoerenti. ---


<a id="3.3"></a>
### 3.3 Controllo Iniziale
Come per la tabella precedente, eseguiamo una rapida query per ispezionare le prime righe della tabella `daily_prices` e assicurarci che il popolamento sia avvenuto correttamente.

In [24]:
%%sql
SELECT * FROM daily_prices LIMIT 10;

Running query in 'sqlite:///ita100_stock_and_macro.db'

id,security_id,price_date,open,high,low,close,volume
1,1,2002-01-01,0.7815925478935242,0.7815925478935242,0.7815925478935242,0.7815925478935242,0
2,1,2002-01-02,0.7781417552006907,0.7919446993003155,0.7643387288290295,0.7729656100273132,1635000
3,1,2002-01-03,0.7764162157245398,0.783317686615885,0.767789335974347,0.7746908068656921,1439500
4,1,2002-01-04,0.7798669742477159,0.7833177097955012,0.7591625609610038,0.7626132965087891,1599000
5,1,2002-01-07,0.7608879315463392,0.7712401387876192,0.7539864600521524,0.7591626048088074,1949500
6,1,2002-01-08,0.7539865525905294,0.7643387611023581,0.7539865525905294,0.7574372887611389,2237500
7,1,2002-01-09,0.7677893922537025,0.7677893922537025,0.7522611227459569,0.7608879208564758,2746500
8,1,2002-01-10,0.7557119158855283,0.7643387146517622,0.7470850348472595,0.7470850348472595,2038500
9,1,2002-01-11,0.7557117125382797,0.7557117125382797,0.7298311586594595,0.7384580373764038,3268000
10,1,2002-01-14,0.7350074720777183,0.7367327988563255,0.7281059181471943,0.7315566539764404,1326609


<a id="4."></a>
## 4. Tabella 3: Dati Macroeconomici (`macro_data`)
Nessuna azienda opera in un vuoto. Per costruire un modello predittivo robusto, è essenziale fornire un contesto macroeconomico. Questa tabella funge da "barometro" delle condizioni di mercato, contenendo una selezione curata di indicatori chiave che influenzano l'economia e i mercati finanziari.
<a id="4.1"></a>
### 4.1 Selezione degli Indicatori Macroeconomici
La selezione dei dati è stata un processo iterativo e una delle sfide principali di questa fase. Dopo aver riscontrato l'inaffidabilità e la scarsa profondità storica di alcune fonti, abbiamo consolidato la nostra pipeline di dati utilizzando esclusivamente la **FRED (Federal Reserve Economic Data)**, una delle fonti più autorevoli e complete al mondo.

Gli indicatori sono stati scelti per catturare diverse dimensioni del rischio e dell'opportunità:
* **Rischio Sovrano e Tassi**: Rendimenti dei titoli di stato (BTP, Bund) e tassi della BCE.

* **Salute Economica**: Tassi di crescita del PIL e indicatori di fiducia (PMI), sia per l'Eurozona che specifici per l'Italia.

* **Pressioni Inflazionistiche**: Tassi di inflazione per l'Eurozona e l'Italia.

* **Sentiment Globale e Fattori Esterni**: Indice di volatilità VIX, prezzo del Petrolio Brent e tasso di cambio EUR/USD.

Questa combinazione di dati locali e globali fornisce al modello una visione a 360 gradi.

In [14]:
# Definiamo le date per il nostro storico macro
start_date = "2002-01-01"
end_date = datetime.datetime.now()

# Dizionario finale con i codici FRED verificati e corretti
fred_series_map = {
    # Dati Globali
    'brent_oil_price': 'DCOILBRENTEU',
    'eur_usd_exchange_rate': 'DEXUSEU',
    'vix_close': 'VIXCLS',
    # Dati di Rischio e Tassi EU/Italia
    'btp_10y_yield': 'IRLTLT01ITM156N',
    'bund_10y_yield': 'IRLTLT01DEM156N',
    'ecb_deposit_rate': 'ECBDFR',
    # Dati Economici Eurozona (EU19)
    'eu_inflation_index': 'CP0000EZ19M086NEST',
    'eu_gdp_value': 'CLVMNACSCAB1GQEA19',
    'eu_pmi_manufacturing': 'BSCICP02EZM460S',
    # Dati Economici Italia
    'it_inflation_index': 'CP0000ITM086NEST',
    'it_gdp_value': 'CLVMNACSCAB1GQIT',
    'it_pmi_manufacturing': 'BSCICP02ITM460S'
}

<a id="4.2"></a>
### 4.2 Popolamento e Armonizzazione dei Dati Macro
La sfida principale nel gestire i dati macroeconomici è la loro **frequenza mista** (giornaliera, mensile, trimestrale). Per creare una tabella coerente, la nostra funzione di popolamento implementa un processo robusto in più fasi:
1. **Download**: Tutte le serie vengono scaricate da FRED.

2. **Armonizzazione**: Viene applicata una tecnica di `forward fill`. Questo propaga l'ultimo valore valido di un indicatore a bassa frequenza (come il PIL) sui giorni successivi, creando una serie giornaliera completa e rispecchiando la disponibilità dell'informazione nel mondo reale.

3. **Feature Engineering**: I dati grezzi (come gli indici di inflazione e i valori assoluti del PIL) vengono trasformati in segnali più potenti per il nostro modello, calcolando i **tassi di variazione anno su anno (YoY)**. Viene inoltre calcolato lo **spread BTP-BUND**.

4. **Pulizia**: Le colonne originali e le righe iniziali con dati nulli (dovute ai calcoli YoY) vengono rimosse per garantire un dataset finale pulito.

5. **Inserimento**: L'intero DataFrame finale viene caricato nel database, sostituendo la tabella `macro_data` per garantire che sia sempre aggiornata all'ultima esecuzione.

In [15]:
def populate_macro_data():
    """
    Popola la tabella 'macro_data' scaricando dati da FRED, eseguendo la
    feature engineering e gestendo le frequenze miste.
    """
    print("Inizio processo di popolamento della tabella 'macro_data'...")
    all_series = []

    # 1. Download dei Dati
    for col_name, series_id in tqdm(fred_series_map.items(), desc="Download FRED"):
        try:
            series = pdr.get_data_fred(series_id, start=start_date, end=end_date)
            series.rename(columns={series_id: col_name}, inplace=True)
            all_series.append(series)
        except Exception as e:
            print(f"  - ATTENZIONE: Impossibile scaricare {series_id}. Errore: {e}")

    # 2. Unione e Armonizzazione
    if not all_series:
        print("Nessun dato scaricato. Processo interrotto.")
        return
    macro_df = pd.concat(all_series, axis=1)
    macro_df.sort_index(inplace=True)
    macro_df.ffill(inplace=True)
    print("\nArmonizzazione delle frequenze tramite forward fill completata.")

    # 3. Feature Engineering
    macro_df['eu_inflation_rate'] = macro_df['eu_inflation_index'].pct_change(periods=365) * 100
    macro_df['it_inflation_rate'] = macro_df['it_inflation_index'].pct_change(periods=365) * 100
    macro_df['eu_gdp_growth'] = macro_df['eu_gdp_value'].pct_change(periods=365) * 100
    macro_df['it_gdp_growth'] = macro_df['it_gdp_value'].pct_change(periods=365) * 100
    macro_df['btp_bund_spread'] = macro_df['btp_10y_yield'] - macro_df['bund_10y_yield']
    
    columns_to_drop = [
        'eu_inflation_index', 'it_inflation_index', 'eu_gdp_value',
        'it_gdp_value', 'bund_10y_yield'
    ]
    macro_df.drop(columns=columns_to_drop, inplace=True)
    macro_df.dropna(inplace=True)
    print("Feature engineering completata.")

    # 4. Inserimento nel Database
    try:
        conn = sqlite3.connect(DB_NAME)
        df_to_insert = macro_df.reset_index()
        date_column_name = df_to_insert.columns[0]
        df_to_insert.rename(columns={date_column_name: 'data_date'}, inplace=True)
        df_to_insert['data_date'] = pd.to_datetime(df_to_insert['data_date']).dt.strftime('%Y-%m-%d')
        
        df_to_insert.to_sql('macro_data', conn, if_exists='replace', index=False)
        
        print(f"\nInserite {len(df_to_insert)} righe nella tabella 'macro_data'.")
        print(f"Colonne finali: {df_to_insert.columns.tolist()}")

    except Exception as e:
        print(f"Errore durante l'inserimento nel database: {e}")
    finally:
        if conn:
            conn.close()
            print("Connessione al database chiusa.")

In [16]:
# Eseguiamo la funzione per creare e popolare la tabella
populate_macro_data()

<a id="4.3"></a>
### 4.3 Controllo Iniziale
Verifichiamo che la tabella sia stata creata e popolata correttamente, ispezionando le ultime righe per assicurarci che contenga i dati più recenti.

In [17]:
%%sql
SELECT * FROM macro_data ORDER BY data_date DESC LIMIT 10;

Running query in 'sqlite:///ita100_stock_and_macro.db'

data_date,brent_oil_price,eur_usd_exchange_rate,vix_close,btp_10y_yield,ecb_deposit_rate,eu_pmi_manufacturing,it_pmi_manufacturing,eu_inflation_rate,it_inflation_rate,eu_gdp_growth,it_gdp_growth,btp_bund_spread
2025-10-17,63.0,1.1613,25.31,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-16,63.0,1.1613,25.31,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-15,63.0,1.1613,20.64,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-14,63.0,1.1613,20.81,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-13,64.15,1.1613,19.03,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-12,64.41,1.1613,21.66,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-11,64.41,1.1613,21.66,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-10,64.41,1.1613,21.66,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-09,67.23,1.1559,16.43,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699
2025-10-08,67.42,1.1606,16.3,3.559,2.0,-10.3,-7.8,1.8741633199464536,1.4586709886547755,0.6751438571300783,0.23482072506277873,0.8656666666666699


<a id="5."></a>
## 5. Quality Assurance Finale del Database
Ora che tutte le tabelle sono popolate, eseguiamo un audit di quality assurance finale. Questo passaggio cruciale verifica l'integrità logica dei dati e la coerenza strutturale dell'intero database. Il nostro obiettivo è certificare che il database sia una base affidabile per tutte le analisi future.
<a id="5.1"></a>
### 5.1 Test di Integrità Logica (Sanity Checks)
Un "sanity check" cerca dati che, sebbene sintatticamente corretti, sono logicamente impossibili nel mondo reale. Eseguiremo una query per verificare le "leggi fisiche" dei dati di mercato all'interno della nostra tabella `daily_prices`. Nello specifico, controlleremo eventuali istanze in cui il massimo del giorno (`high`) sia registrato come inferiore al prezzo di apertura (`open`) o di chiusura (`close`), un chiaro segno di corruzione dei dati.

In [18]:
%%sql
SELECT
    SUM(CASE WHEN high < low THEN 1 ELSE 0 END) AS high_minore_di_low,
    SUM(CASE WHEN high < open THEN 1 ELSE 0 END) AS high_minore_di_open,
    SUM(CASE WHEN high < close THEN 1 ELSE 0 END) AS high_minore_di_close
FROM daily_prices;

Running query in 'sqlite:///ita100_stock_and_macro.db'

high_minore_di_low,high_minore_di_open,high_minore_di_close
None,None,None


<a id="5.2"></a>
### 5.2 Test di Coerenza Strutturale
Qui verificheremo che le relazioni tra le nostre tabelle siano implementate correttamente e che non ci siano record "orfani".

Per prima cosa, testeremo la relazione `FOREIGN KEY` eseguendo una `JOIN` tra le tabelle `securities_master` e `daily_prices`. Se questa query viene eseguita senza errori e restituisce dati, conferma che le tabelle sono collegate correttamente.

In [19]:
%%sql
SELECT
    sm.ticker,
    dp.price_date,
    dp.close
FROM
    daily_prices AS dp
JOIN
    securities_master AS sm ON dp.security_id = sm.id
LIMIT 10;

Running query in 'sqlite:///ita100_stock_and_macro.db'

ticker,price_date,close


Successivamente, eseguiremo un test più rigoroso per assicurarci che non ci siano record "orfani", ovvero voci di prezzo in `daily_prices` che non corrispondono a nessun titolo valido nella nostra tabella `securities_master`. La seguente query dovrebbe restituire zero righe, confermando l'integrità referenziale del database.

In [20]:
%%sql
SELECT
    dp.security_id,
    COUNT(*) as numero_record_orfani
FROM
    daily_prices dp
LEFT JOIN
    securities_master sm ON dp.security_id = sm.id
WHERE
    sm.id IS NULL
GROUP BY
    dp.security_id;

Running query in 'sqlite:///ita100_stock_and_macro.db'

security_id,numero_record_orfani


<a id="6."></a>
## 6. Creazione dello Script di Aggiornamento
Un database finanziario non è un artefatto statico, ma un organismo vivente che necessita di essere costantemente aggiornato con i dati più recenti. Questo notebook ha avuto lo scopo di eseguire il **"bootstrap"** del database, ovvero la sua creazione iniziale e il popolamento massivo con dati storici.

Per la manutenzione futura, tuttavia, non è efficiente rieseguire l'intero notebook. La best practice consiste nel creare uno **script di aggiornamento** separato, leggero e automatizzabile.
<a id="6.1"></a>
### 6.1 Logica e Best Practice
Lo script `update_database.py` è stato progettato per essere eseguito periodicamente (es. una volta al mese) per mantenere il database allineato con i nuovi dati di mercato. La sua logica si basa su due approcci distinti:
- **Aggiornamento Incrementale per i Prezzi**: Per la tabella `daily_prices`, lo script è "intelligente". Per ogni titolo, interroga il database per trovare l'ultima data disponibile e scarica da `yfinance` solo i dati successivi a quella data. Questo processo è estremamente veloce ed efficiente.

- **Ricostruzione Completa per i Dati Macro**: Per la tabella `macro_data`, l'approccio più robusto è quello di ricreare l'intera tabella ad ogni esecuzione. Questo garantisce che i calcoli complessi (come il forward fill e le variazioni percentuali) siano sempre eseguiti sull'intero set di dati, evitando inconsistenze che potrebbero derivare da un aggiornamento parziale.

Lo script è stato inoltre reso **modulare**, con funzioni separate per ogni compito, e **robusto**, includendo gli stessi controlli di qualità che abbiamo definito durante la creazione del database.
<a id="6.2"></a>
### 6.2 Codice dello Script `update_database.py`
Di seguito è riportato il codice completo dello script di aggiornamento. Questo codice non va eseguito all'interno del notebook, ma salvato in un file separato chiamato `update_database.py` nella stessa cartella del database. Potrà poi essere eseguito da un terminale o da un task scheduler per automatizzare il processo di aggiornamento.

<a id="7."></a>
## 7. Nota Metodologica
La realizzazione di questo progetto si è basata su un approccio di sviluppo moderno, dove l'analista agisce come **architetto e domain expert**, e un modello AI avanzato funge da **"pair programmer"** per accelerare l'implementazione.

In questo framework, **Google Gemini 2.5 Pro** è stato utilizzato come partner per:
* **Accelerare la Scrittura del Codice**: Generare codice standard (boilerplate) e funzioni complesse basate su specifiche dettagliate.

* **Facilitare il Debugging**: Identificare rapidamente errori di sintassi e logici.

* **Esplorare Best Practice**: Suggerire approcci e librerie standard del settore per compiti specifici.

**La responsabilità della metodologia, del design del database, della strategia di analisi, del debugging critico e di tutte le decisioni strategiche è rimasta interamente dell'autore**. Questo approccio ibrido ha permesso di concentrare le energie non sulla sintassi, ma sui problemi di livello superiore: la qualità dei dati, la logica del modello e l'interpretazione dei risultati.